In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader

In [9]:
def encode_pdf(path,chunk_size=1000,chunk_overlap=200):
    """ 
    This method will chunk a pdf and then convert the chunks into embedding and store them into a vector database

    Args:
        path: path to the pdf file
        chunk_size: paragraph length of each chunk
        chunk_overlap: max amount of consicutive overlap between chunks

    Return:
        A fassi vector store containing the encoded pdf content
    """

    loader = PyPDFLoader(path)
    docs=loader.load()
    print(docs[0])


In [11]:
doc_path = "harry_potter_1.pdf"
encode_pdf(doc_path)

ImportError: `pypdf` package not found, please install it with `pip install pypdf`